In [43]:
# %%writefile trade_model.py
import numpy as np
import pandas as pd
from khayyam import JalaliDate, JalaliDatetime
import pystore



def add_diff_min_max(df): 
    df["diff_min_max"] = (df['max']-df['min'])*100/(df['min'])

def add_diff_ending(df):
    df["diff_open"] = (df['lastday']-df['ending'])*100/(df['lastday'])

def add_adjust_scale(df_symbol):
    lastdays = df_symbol["lastday"].copy().drop(df_symbol.index[0])
    endings = df_symbol["ending"].copy().drop(df_symbol.index[-1])
    endings.index = lastdays.index
    scale = lastdays/endings
    scale[df_symbol.index[0]] = 1
    df_symbol.loc[:, "adj_scale"] = scale
    
def add_adjust(df):
    adj = df.loc[df["adj_scale"] < 1].index
    df["adj_open"] = df["open"]
    df["adj_close"] = df["close"]
    df["adj_ending"] = df["ending"]
    df["adj_min"] = df["min"]
    df["adj_max"] = df["max"]
    adj_headers = ["adj_min", "adj_max", "adj_close", "adj_open", "adj_ending"]
    for date in adj:
        scale = df.loc[date, "adj_scale"]
        df.loc[df.index[0]:date, adj_headers] = df.loc[df.index[0]:date, adj_headers].transform(lambda x: x * scale)

def add_log_adj(df):
    adj = df.loc[df["adj_scale"] < 1].index
    df["log_adj_open"] = np.log10(df["adj_open"])
    df["log_adj_close"] = np.log10(df["adj_close"])
    df["log_adj_ending"] = np.log10(df["adj_ending"])
    df["log_adj_min"] = np.log10(df["adj_min"])
    df["log_adj_max"] = np.log10(df["adj_max"])

class DataModel:
    TA_SYMBOLS = ["خپارس", "خكاوه", "فاسمين", "شبريز", "ونوين", "كنور", "ثشرق", "كاما", "ورنا", "خمحركه", "دامين",
                  "خاور", "خپارس", "خودرو", "فجام", "وبصادر"]

    def __init__(self,data_location, file_names=[]):  
        self.data_location = data_location;
        self.file_names = file_names;
        pystore.set_path('/home/nimac/.pystore')
        self.store = pystore.store('tradion_store')
        self.collection = self.store.collection('boors')

    def __read_csv(self, file_name):
        return pd.read_csv(f'{self.data_location}/{file_name}', sep=',',header=[0], parse_dates=["date"])

    def read(self):
        dfs = []
        for name in self.file_names:
            dfs.append(self.__read_csv(name))
        self.df = pd.concat(dfs, ignore_index=True)
        add_diff_min_max(self.df)
        add_diff_ending(self.df)
        self.df = self.df.set_index('date')
        
#         print("hi")
#         self.df = self.df.groupby("symbol").apply(add_adjust_scale)
#         self.allSymbols = self.df.symbol.tolist()
#         self.symbols = list(set(self.df.symbol))[1:]
#         for symbol in self.symbols:
#         counts = Counter(self.allSymbols)
#         testSymbols = []
#         tmpSymbols = []
#         for symbol in symbols:
#             if counts[symbol] > RECORD_THRESHOLD:
#                 tmpSymbols.append(symbol)
#         for i in range(TESTCASE_NUMBER):
#             ran = random.randint(0, len(tmpSymbols)-1)
#             testSymbols.append(tmpSymbols[ran])
#             tmpSymbols.remove(tmpSymbols[ran])
#         print("test symbol", len(testSymbols))

    def store_in_pystore(self):
        self.collection.write('ALL', self.df, metadata={'source': 'tsetmc'}, overwrite=True)
    
    def restore_from_pystore(self):
        self.item = collection.item('ALL')
        self.df = item.to_pandas()
    
    def get(self, symbol, start="", end=""):
        
        if start == "":
            start = self.df.index[0]
        else:
            s_date = start.split("-")
            start = JalaliDate(s_date[0], s_date[1], s_date[2]).todate()
        if end == "":
            end = self.df.index[-1]
        else:
            e_date = end.split("-")
            end = JalaliDate(e_date[0], e_date[1], e_date[2]).todate()
        tmpdf = self.df.loc[start:end]
        df = tmpdf.loc[tmpdf["symbol"]==symbol].copy()
        add_adjust_scale(df)
        add_adjust(df)
        add_log_adj(df)
        return df
    
    def check_contains_name(self, symbol):
        dm.df.loc[dm.df["symbol"].str.contains(symbol)==True]
        
    def get_overal_corr(self, symbols):
        df_corr = pd.DataFrame()
        for symbol in symbols:
            df_corr[f'{symbol}_log_adj_ending'] = self.get(symbol)["log_adj_ending"]
        return df_corr.corr()

Overwriting trade_model.py


In [39]:
dm = DataModel("../xcels", ["master0.csv", "master1.csv"])
dm.read()

In [11]:
symbol = "خساپا"
df = dm.get(symbol,"1398-4-28", "1398-5-2")
# df = dm.get(symbol)
add_adjust_scale(df)
abnormal = df[df["adj_scale"] < 1]
for a in abnormal.index:
    print(JalaliDate(a), abnormal.loc[a, "adj_scale"])
add_adjust(df)

df.loc[:, ["close", "open", "ending", "lastday"]]
df

,symbol,name,amount,volume,value,lastday,open,close,last-change,last-percent,...,adj_open,adj_close,adj_ending,adj_min,adj_max,log_adj_open,log_adj_close,log_adj_ending,log_adj_min,log_adj_max
date,,,,,,,,,,,,,,,,,,,,,
2019-07-20,خساپا,سايپا,12129.0,364203156.0,5.700689e+11,1526.0,1510.0,1499.0,-27.0,-1.77,...,1510.0,1499.0,1565.0,1490.0,1594.0,3.178977,3.175802,3.194514,3.173186,3.202488
2019-07-24,خساپا,سايپا,2466.0,41511494.0,6.369656e+10,1565.0,1545.0,1520.0,-45.0,-2.88,...,1545.0,1520.0,1534.0,1520.0,1558.0,3.188928,3.181844,3.185825,3.181844,3.192567


In [ ]:
import trade_model as td
dm = td.DataModel("../xcels", ["master0.csv", "master1.csv"])
dm.read()
symbol = "فولاد"
df = dm.get(symbol, "1394-4-23", "1394-6-18")
df